imprements by https://www.kaggle.com/code/awsaf49/birdclef23-effnet-fsr-cutmixup-train

In [ ]:
import sys
sys.path.append('/kaggle/input/birdclef-2023-baseline')
sys.path.append('/kaggle/input/torchlibrosa-master')
sys.path.append('/kaggle/input/tinytag-master')

# Impoert library

In [1]:
import os
import random
from glob import glob
from pathlib import Path
import yaml
from tqdm import tqdm
import IPython.display as ipd

import numpy as np
import pandas as pd

import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt

import librosa
import wandb
from sklearn import metrics

import torch
from torch import nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler

from timm.scheduler import CosineLRScheduler

In [2]:
from config import CFG
from pytorch_model import BirdCLEF23Net
import pytorch_modeler as modeler
import pytorch_preprocessing as prep
import common as com

In [3]:
modeler.set_seed(CFG.seed)
# setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print('Debug :', CFG.debug)

cuda:0
Debug : False


In [4]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを設定
jst = pytz.timezone('Asia/Tokyo')

# 現在時刻を取得し、日本時間に変換
now = datetime.now(jst)

# 現在時刻を文字列に変換
now_str = now.strftime('%Y-%m-%d %H:%M:%S')

print(now_str)

2023-04-09 21:59:11


# Data Frame

In [5]:
test_paths = glob('/kaggle/input/birdclef-2023/test_soundscapes/*ogg')
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df['filename'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.ogg',''))
test_df.head(2)

,filepath,filename
0,/kaggle/input/birdclef-2023/test_soundscapes/s...,soundscape_29201


# Inference

In [6]:
test_ds = prep.BirdDataset(test_df, is_train=False, is_test=True)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_ds,
    batch_size=CFG.test_bs,
    shuffle=False,
    num_workers = os.cpu_count(),
    collate_fn = modeler.inference_collate,
    )

In [7]:
num_classes = CFG.num_classes
   
# Compute batch size and number of samples to drop
infer_bs = CFG.test_bs
model = BirdCLEF23Net(num_classes=CFG.num_classes, pretrained=False)
    
# Test
print('# Loading model')
model.load_state_dict(torch.load(CFG.inference_model_path, map_location=device), strict=True)
model.to(device)

# Predict on the validation data for oof result
print('# Infering Test')
model.eval()
test_pred = []
row_id = []
with torch.no_grad():
    for inputs, sample_info in tqdm(test_dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = torch.softmax(outputs, dim=1)
        test_pred.append(outputs.detach().cpu().numpy())
        row_id.extend(sample_info['row_id'])

test_pred = np.concatenate(test_pred, axis=0)

# Loading best model
# Infering Test


100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


In [8]:
bird_names = CFG.class_names
sub_cols = ['row_id'] + bird_names
sub_df = pd.DataFrame(columns=sub_cols)
sub_df['row_id'] = row_id
sub_df[bird_names] = test_pred

In [9]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.000071,0.000076,0.000361,3.111759e-05,0.000053,0.000182,0.000039,0.000014,0.028085,...,2.086845e-06,0.000010,0.000231,0.000026,6.299696e-06,0.000163,6.651048e-05,0.000005,2.622424e-05,4.636667e-05
1,soundscape_29201_10,0.000208,0.023042,0.000381,3.605192e-03,0.000049,0.000166,0.001888,0.000174,0.000367,...,2.030679e-06,0.000014,0.015136,0.000183,7.010020e-05,0.000313,4.017679e-04,0.000004,4.363204e-04,4.195240e-04
2,soundscape_29201_15,0.003267,0.000447,0.000429,1.008071e-04,0.000025,0.000116,0.000088,0.000227,0.000110,...,1.686499e-06,0.000013,0.001231,0.000144,3.449673e-05,0.000169,4.644011e-05,0.000074,7.103547e-05,3.102078e-04
3,soundscape_29201_20,0.000425,0.000221,0.000438,1.804139e-04,0.000049,0.000024,0.000014,0.000152,0.000342,...,9.810990e-07,0.000020,0.000343,0.000404,6.009940e-05,0.000048,7.436436e-06,0.000009,1.628175e-05,6.473296e-06
4,soundscape_29201_25,0.000072,0.000034,0.000426,1.055006e-04,0.000247,0.000321,0.000150,0.000452,0.000933,...,1.537537e-06,0.000008,0.000700,0.000648,5.475166e-05,0.000086,2.028847e-05,0.000114,1.545866e-04,1.127970e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.000045,0.000012,0.000010,9.616911e-07,0.000073,0.000003,0.000083,0.000075,0.000035,...,1.148555e-07,0.000003,0.000990,0.000017,1.369850e-05,0.000119,1.225729e-06,0.000004,1.929499e-06,6.878817e-07
116,soundscape_29201_585,0.000671,0.001722,0.000322,3.044705e-05,0.000221,0.000243,0.078395,0.000128,0.000044,...,9.128436e-06,0.000009,0.005298,0.000171,4.975317e-04,0.003574,1.004683e-04,0.000022,3.391971e-05,2.244076e-05
117,soundscape_29201_590,0.000157,0.002315,0.000538,1.321835e-05,0.000300,0.000307,0.000800,0.000091,0.000068,...,1.278939e-06,0.000003,0.027835,0.000040,1.754176e-04,0.000115,3.518908e-05,0.000059,3.705721e-05,1.562963e-05
118,soundscape_29201_595,0.000004,0.000028,0.000001,3.236694e-06,0.000006,0.000004,0.000033,0.000128,0.000002,...,5.217959e-09,0.000009,0.000526,0.000587,7.348664e-07,0.000022,3.492594e-07,0.000005,2.003218e-07,6.401017e-07


In [10]:
sub_df.to_csv('submission.csv',index=False)